# Using the Python SDK for AutoML with the EON Tuner

<!--- Do not modify the markdown for this example directly! It is generated from a notebook in https://github.com/edgeimpulse/notebooks --->

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://docs.edgeimpulse.com/docs/tutorials/ml-and-data-engineering/ei-python-sdk/python-sdk-eon-tuner"><img src="https://raw.githubusercontent.com/edgeimpulse/notebooks/main/.assets/images/logo-ei-32px.png" /> View on edgeimpulse.com</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/edgeimpulse/notebooks/blob/main/notebooks/python-sdk-eon-tuner.ipynb"><img src="https://raw.githubusercontent.com/edgeimpulse/notebooks/main/.assets/images/logo-colab-32px.png" /> Run in Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/edgeimpulse/notebooks/blob/main/notebooks/python-sdk-eon-tuner.ipynb"><img src="https://raw.githubusercontent.com/edgeimpulse/notebooks/main/.assets/images/logo-github-32px.png" /> View source on GitHub</a>
  </td>
  <td>
    <a href="https://raw.githubusercontent.com/edgeimpulse/notebooks/main/notebooks/python-sdk-eon-tuner.ipynb" download><img src="https://raw.githubusercontent.com/edgeimpulse/notebooks/main/.assets/images/icon-download-32px.png" /> Download notebook</a>
  </td>
</table>

The [EON Tuner](https://docs.edgeimpulse.com/docs/edge-impulse-studio/eon-tuner) is Edge Impulse's automated machine learning (AutoML) tool to help you find the best combination of blocks and hyperparameters for your model and within your hardware constraints. This example will walk you through uploading data, running the EON Tuner, and interpreting the results.

> **WARNING:** This notebook will add and delete data in your Edge Impulse project, so be careful! We recommend creating a throwaway project when testing this notebook.

To start, create a new project in Edge Impulse. Do not add any data to it.

In [ ]:
# If you have not done so already, install the following dependencies
# !python -m pip install edgeimpulse

In [ ]:
from edgeimpulse.experimental.data import (
    upload_directory
)
from edgeimpulse.experimental.tuner import (
    start_tuner,
    check_tuner,
    tuner_report,
    set_impulse_from_trial,
)

You will need to obtain an API key from an Edge Impulse project. Log into [edgeimpulse.com](https://edgeimpulse.com/) and create a new project. Open the project, navigate to **Dashboard** and click on the **Keys** tab to view your API keys. Double-click on the API key to highlight it, right-click, and select **Copy**.

![Copy API key from Edge Impulse project](https://raw.githubusercontent.com/edgeimpulse/notebooks/main/.assets/images/python-sdk-copy-ei-api-key.png)

Note that you do not actually need to use the project in the Edge Impulse Studio. We just need the API Key.

Paste that API key string in the `ei.API_KEY` value in the following cell:

In [ ]:
# Settings
ei.API_KEY = "ei_dae2..." # Change this to your Edge Impulse API key

In [ ]:
### ~~~---...___DELETE ME!!!___...---~~~
ei.API_KEY="ei_2fad0f493031e0a1513d04d80916ab6a4db7ef0ad88fdbb1bd9147a92979b3ba" # replace with your key under dashboard/keys
ei.API_ENDPOINT="http://localhost:4800/v1"
ei.INGESTION_ENDPOINT="http://localhost:4810"

## Upload dataset

We start by downloading the [continuous motion dataset](https://docs.edgeimpulse.com/docs/pre-built-datasets/continuous-gestures) and uploading it to our project.

In [ ]:
# Download and unzip gesture dataset
!mkdir -p dataset/
!wget -P dataset -q https://cdn.edgeimpulse.com/datasets/gestures.zip
!unzip -q dataset/gestures.zip -d dataset/gestures/

In [ ]:
# Upload training dataset
resp = upload_directory(
    directory="dataset/gestures/training",
    category="training",
)
print(f"Uploaded {len(resp.successes)} training samples")

# Upload test dataset
resp = upload_directory(
    directory="dataset/gestures/testing",
    category="testing",
)
print(f"Uploaded {len(resp.successes)} testing samples")

In [ ]:
# Uncomment the following if you want to delete the temporary dataset folder
#!rm -rf dataset/

## Run the Tuner

We start the tuner with `check_tuner()` and wait for completion via `check_tuner()`. In this example, we configure the tuner to target for the *alif-hp device*. Since we want to classify the motion, we choose a `classifcation_type` of `classification` and our dataset as motion continous. We constrain our model to a latency of 100ms for running the impulse. 

> **NOTE:** We set the max trials to 3 here. In a real life situation, you will omit this so the tuner decides the best number of trials.

Once the tuner is done, you can print out the results to determine the best combination of blocks and hyperparameters.

In [ ]:
# Start tuner
start_tuner(
    target_device="alif-hp",
    classification_type="classification",
    dataset_category="motion_continuous",
    target_latency=100,
    tuning_max_trials=3,
)

# Wait while checking the tuner's progress. This could take a few minutes.
state = check_tuner(
    wait_for_completion=True
)

## Print EON Tuner results

The configuration settings and output of the EON Tuner is stored in the variable `state`. You can access the results of the various trials with `state.trials`. Note that some trials can fail, so it's a good idea to test the status of each trial.

From there, you will want to sort the results based on some metric. In this example, we will sort based on float32 test set accuracy from highest to lowest.

> Note: Edge Impulse supports only one learning block per project at this time (excluding anomaly detection blocks). As a result, we will use the first learning block (e.g. `learning_blocks[0]`) in the list to extract metrics.

In [ ]:
import json

In [ ]:
# Set quantization ("float32" or "int8")
qtzn = "int8"

# Filter out all failed trials
results = [r for r in state.trials if r.status == "completed"]

# Extract float32 accuracies from the trial results
accuracies = []
for result in results:
    accuracy = result.impulse.learn_blocks[0]["metrics"]["test"][qtzn]["accuracy"]
    accuracies.append(accuracy)

# Sort the results based on float32 accuracies
acc_results = zip(accuracies, results)
sorted_results = sorted(acc_results, reverse=True, key=lambda x: list(x)[0])
sorted_results = [result for _, result in sorted_results]

Now that we have the sorted results, we can extract the values we care about. We will print out the following metrics along with the impulse configuration (processing/learning block configuration and hyperparameters) of the top-performing trial.

This will help you determine if the impulse can fit on your target hardware and run fast enough for your needs. The impulse configuration can be used to recreate the processing and learning blocks on Edge Impulse. That being said, you will not need to do so manually! Later, we will set the project impulse based on the trial ID to simply deploy (rather than re-train).

> **Note:** we assume the first learning block has the metrics we care about.

In [ ]:
print(json.dumps(qtzn_metadata[0]["memory"], indent=2))

In [ ]:
learn_metadata = sorted_results[trial_idx].impulse.learn_blocks[0]["metadata"]
qtzn_metadata = [m for m in learn_metadata["modelValidationMetrics"] if m.get("type") == qtzn]
tflite_ram = qtzn_metadata[0]["memory"]["tflite"]["ram"]
tflite_rom = qtzn_metadata[0]["memory"]["tflite"]["rom"]
eon_ram = qtzn_metadata[0]["memory"]["eon"]["ram"]
eon_rom = qtzn_metadata[0]["memory"]["eon"]["rom"]

In [ ]:
# The top performing impulse is the first element
# Note that "top performing" is the highest float32 accuracy on the test set
trial_idx = 0

# Print info about the processing (DSP) blocks and store RAM usage
dsp_ram = 0
print("Processing blocks")
print("===")
for i, dsp_block in enumerate(sorted_results[trial_idx].impulse.dsp_blocks):
    print(f"Processing block {i}")
    print("---")
    print("Block:")
    print(json.dumps(dsp_block["block"], indent=2))
    print("Config:")
    print(json.dumps(dsp_block["config"], indent=2))
    dsp_ram += dsp_block["estimatedPerformance"]["ram"]
print()

# Print info about the learning blocks and store RAM/ROM usage
tflite_ram = 0
tflite_rom = 0
eon_ram = 0
eon_rom = 0
print("Learning blocks")
print("===")
for i, learn_block in enumerate(sorted_results[trial_idx].impulse.learn_blocks):
    print(f"Learn block {i}")
    print("---")
    print("Block:")
    print(json.dumps(learn_block["block"], indent=2))
    print("Config:")
    print(json.dumps(learn_block["config"], indent=2))
    metadata = learn_block["metadata"]
    qtzn_metadata = [m for m in metadata["modelValidationMetrics"] if m.get("type") == qtzn]
    tflite_ram += qtzn_metadata[0]["memory"]["tflite"]["ram"]
    tflite_rom += qtzn_metadata[0]["memory"]["tflite"]["rom"]
    eon_ram += qtzn_metadata[0]["memory"]["eon"]["ram"]
    eon_rom += qtzn_metadata[0]["memory"]["eon"]["rom"]
print()

# Print the metrics
print(f"Metrics ({qtzn})")
print("===")
metrics = sorted_results[trial_idx].impulse.learn_blocks[0]["metrics"]
print(f"Validation accuracy: {metrics['validation'][qtzn]['accuracy']}")
print(f"Test accuracy: {metrics['test'][qtzn]['accuracy']}")
print(f"Estimated processing blocks RAM (bytes): {dsp_ram}")
print(f"Estimated learning blocks RAM (bytes): {tflite_ram}")
print(f"Estimated learning blocks ROM (bytes): {tflite_rom}")
print(f"Estimated learning blocks RAM with EON Compiler (bytes): {eon_ram}")
print(f"Estimated learning blocks ROM with EON Compiler (bytes): {eon_rom}")
print()

### TODO: PRINT LATENCY ESTIMATES!!!

## Results as a DataFrame

If you have [pandas](https://pandas.pydata.org/) installed, you can make the previous section much easier by reporting metrics as a DataFrame.

In [ ]:
# Convert the state metrics into a DataFrame
df = tuner_report(state)
df.head()

In [ ]:
# Sort the DataFrame by validation (int8) accuracy and print out best trial
df = df.sort_values(by='val_int8_accuracy', ascending=False)
df.iloc[0]

## Set trial as impulse and deploy

We can replace the current impulse with the top performing trial from the EON Tuner. From there, we can deploy it, just like we would any impulse.

In [ ]:
# Get the trial ID and set that to our impulse
trial_id = df.iloc[0].trial_id
set_impulse_from_trial(trial_id)